In [45]:
"""Parse the CSV file and return a list of MempoolTransactions."""
def parse_csv(filename):
    with open(filename) as f:
        return [line.strip().split(',') for line in f.readlines()]

def create_valid_mempool_list(parsed):
    block_ids = []
    block_fee = []
    block_wts = []
    parent_set = set()
    for x in parsed:
        if x[1].isnumeric() and x[2].isnumeric():
            parent_set.add(x[0])
            if x[3] =="" or parent_set.issuperset(x[3].split(';')):
                block_ids.append(x[0])
                block_fee.append(int(x[1]))
                block_wts.append(int(x[2]))
    return (block_ids, block_fee, block_wts)


In [39]:
import numpy as np

def get_optimal_blocks_dp(values,weights,n_items,capacity=4000000,return_all=False):
    check_inputs(values,weights,n_items,capacity)

    table = np.zeros((n_items+1,capacity+1),dtype=np.float32)
    keep = np.zeros((n_items+1,capacity+1),dtype=np.float32)

    for i in range(1,n_items+1):
        for w in range(0,capacity+1):
            wi = weights[i-1] # weight of current item
            vi = values[i-1] # value of current item
            if (wi <= w) and (vi + table[i-1,w-wi] > table[i-1,w]):
                table[i,w] = vi + table[i-1,w-wi]
                keep[i,w] = 1
            else:
                table[i,w] = table[i-1,w]
    picks = []
    K = capacity
    for i in range(n_items,0,-1):
        if keep[i,K] == 1:
            picks.append(i)
            K -= weights[i-1]
    picks.sort()
    picks = [x-1 for x in picks] # change to 0-index
    if return_all:
        max_val = table[n_items,capacity]
        return (picks,max_val)
    return picks

def check_inputs(values,weights,n_items,capacity):
    # check variable type
    assert(isinstance(values,list))
    assert(isinstance(weights,list))
    assert(isinstance(n_items,int))
    assert(isinstance(capacity,int))
    # check value type
    assert(all(isinstance(val,int) or isinstance(val,float) for val in values))
    assert(all(isinstance(val,int) for val in weights))
    # check validity of value
    assert(all(val >= 0 for val in weights))
    assert(n_items > 0)
    assert(capacity > 0)

In [46]:
temp = parse_csv("test.csv")
block_ids, block_fee, block_wts = create_valid_mempool_list(temp)
optimal_block_list = get_optimal_blocks(block_fee, block_wts, len(block_ids))
result = [block_ids[i] for i in optimal_block_list]

In [60]:
from datetime import datetime
filename = "Blocklist "+str(datetime.utcnow())+".txt"

In [61]:
textfile = open(filename, "w")
for element in result:
    textfile.write(element + "\n")
textfile.close()

In [8]:
from tqdm.auto import tqdm
tnum = 500
loop = tqdm(total = tnum+1, position=0, leave=False)
for i in range(501):
    loop.set_description("Loading...".format(i))
    loop.update(1)
# import time
# with tqdm(total=1000, desc="Adding Users", bar_format="{l_bar}{bar} [ time left: {remaining} ]") as pbar:
#     for i in range(100):
#         time.sleep(3)
#         pbar.update(1)

Loading...: 100%|██████████| 501/501 [00:20<00:00, 680.74it/s]

In [ ]:
# Composition template
# def increment(num): 
#     return num + 1 
 
# def square(num): 
#     return num * num 
 
# nine = square(increment(2)) 
# five = increment(square(2)) 
 
# class NumHooker: 
#     def __init__(self): 
#         self.funclist = list() 
#         self.add(lambda x: x) 
#     def __call__(self, num): 
#         result = num 
#         for each_func in self.funclist: 
#             result = each_func(result) 
#         return result 
#     def add(self, func): 
#         self.funclist.append(func) 
 
# incsquare = NumHooker() 
# incsquare.add(increment) 
# incsquare.add(square) 
 
# five = incsquare(2) 

In [39]:
import utils


class MempoolTransaction():
    def __init__(self, txid, fee, weight):
        self.txid = txid
        self.fee = int(fee)
        self.weight = int(weight)

    def __lt__(self, other):
        return (float(self.fee)/float(self.weight)) > (float(other.fee)/float(other.weight))

# def main():
#     filename = "mempool.csv"
#     parsed = utils.parse_csv(filename)
#     block_ids, block_fee, block_wts = utils.create_valid_mempool_list(parsed)
    

# if __name__ == "__main__":
#     main()


In [40]:
filename = "mempool.csv"
max_block_weight = 4e6
parsed = utils.parse_csv(filename)
block_ids, block_fee, block_wts = utils.create_valid_mempool_list(parsed)
mempool_list = [MempoolTransaction(block_ids[i], block_fee[i], block_wts[i]) for i in range(len(block_ids))]
# mempool_list.sort()

In [44]:
result = []
weight_sum = 0
fee_sum = 0
for transaction in mempool_list:
    if weight_sum + transaction.weight < max_block_weight:
        result.append(transaction.txid)
        weight_sum += transaction.weight
        fee_sum += transaction.fee
    else:
        break
#     if weight_sum > max_block_weight:
#         break
#     else:
#         result.append(transaction.txid)
print(weight_sum)
print(fee_sum)

3999344
5623886


In [46]:
print(mempool_list[0].fee/mempool_list[0].weight)
print(mempool_list[1].fee)
test_block(block_ids, block_fee, block_wts, result)

149.62908011869436
90000
Elements in the block: 3207
Total sum of the elements: 5623886
Total weight of the elements: 3999344


In [45]:
def test_block(mempool_list, values, weights, block_entries):
    value_list = dict(zip(mempool_list, values))
    weight_list = dict(zip(mempool_list, weights))
    sum = 0
    wts = 0
    for i in block_entries:
        sum += value_list[i]
        wts += weight_list[i]
    print("Elements in the block: "+str(len(block_entries)))
    print("Total sum of the elements: "+str(sum))
    print("Total weight of the elements: "+str(wts))
